## Name: Vishal Menon
## Github username: vmeno0020
## USC ID: 3377740336

In [4]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
VertCol_df = pd.read_csv('../data/vertebral+column/column_2C.dat', sep=' ',header=None)
VertCol_df.columns = ['Pelvic Incidence', 'Pelvic Tilt', 'Lumbar Lordosis Angle', 'Sacral Slope', 'Pelvic Radius', 'Grade of Spondylolisthesis', 'Class_Label']
# assign Normal and Abnormal Class Labels as binary values: Normal NO = 0, Abnormal AB = 1
VertCol_df.loc[VertCol_df.Class_Label == 'NO', 'Class_Label'] = 0
VertCol_df.loc[VertCol_df.Class_Label == 'AB', 'Class_Label'] = 1
VertCol_df.Class_Label = VertCol_df.Class_Label.astype(int)
VertCol_df

,Pelvic Incidence,Pelvic Tilt,Lumbar Lordosis Angle,Sacral Slope,Pelvic Radius,Grade of Spondylolisthesis,Class_Label
0,63.03,22.55,39.61,40.48,98.67,-0.25,1
1,39.06,10.06,25.02,29.00,114.41,4.56,1
2,68.83,22.22,50.09,46.61,105.99,-3.53,1
3,69.30,24.65,44.31,44.64,101.87,11.21,1
4,49.71,9.65,28.32,40.06,108.17,7.92,1
...,...,...,...,...,...,...,...
305,47.90,13.62,36.00,34.29,117.45,-4.25,0
306,53.94,20.72,29.22,33.22,114.37,-0.42,0
307,61.45,22.69,46.17,38.75,125.67,-2.71,0
308,45.25,8.69,41.58,36.56,118.55,0.21,0


## Part B

In [ ]:
sns.pairplot(VertCol_df, hue='Class_Label')
plt.show()

In [ ]:
sns.boxplot(x='Class_Label', y='Pelvic Incidence', data=VertCol_df, hue='Class_Label')

In [ ]:
sns.boxplot(x='Class_Label', y='Pelvic Tilt', data=VertCol_df, hue='Class_Label')

In [ ]:
sns.boxplot(x='Class_Label', y='Lumbar Lordosis Angle', data=VertCol_df, hue='Class_Label')

In [ ]:
sns.boxplot(x='Class_Label', y='Sacral Slope', data=VertCol_df, hue='Class_Label')

In [ ]:
sns.boxplot(x='Class_Label', y='Pelvic Radius', data=VertCol_df, hue='Class_Label')

In [ ]:
sns.boxplot(x='Class_Label', y='Grade of Spondylolisthesis', data=VertCol_df, hue='Class_Label')

## Part B iii

In [ ]:
class_0 = VertCol_df[VertCol_df['Class_Label'] == 0]
class_1 = VertCol_df[VertCol_df['Class_Label'] == 1]

# Select 1st 70 rows of Class 0 and 1st 140 rows of Class 1 for training set
train_class_0 = class_0.iloc[:70]
train_class_1 = class_1.iloc[:140]

# combine subsets for training set
training_set = pd.concat([train_class_0, train_class_1])

# remaining data for  test set
test_class_0 = class_0.iloc[70:]
test_class_1 = class_1.iloc[140:]

# combine subsets to form the test set
test_set = pd.concat([test_class_0, test_class_1])

print("Training Set:\n", training_set)
print("Testing Set:\n", test_set)

In [ ]:
# testing and training data
X_train = training_set.drop('Class_Label', axis=1)
y_train = training_set['Class_Label']
X_test = test_set.drop('Class_Label', axis=1)
y_test = test_set['Class_Label']

## Part C i

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

## Part C ii

In [ ]:
k_values = range(208, 0, -3)
train_errors = []
test_errors = []

for k in k_values:
    KNN_EuclideanMetric = KNeighborsClassifier(n_neighbors=k, metric='euclidean') #euclidean metric classifier
    KNN_EuclideanMetric.fit(X_train, y_train)
    
    # Predict on training and test data
    train_pred = KNN_EuclideanMetric.predict(X_train)
    test_pred = KNN_EuclideanMetric.predict(X_test)
    
    # Calculate and record errors
    train_error = 1 - accuracy_score(y_train, train_pred)
    test_error = 1 - accuracy_score(y_test, test_pred)
    train_errors.append(train_error)
    test_errors.append(test_error)
    
plt.figure(figsize=(8, 4))
plt.plot(k_values, train_errors, label='Train Error')
plt.plot(k_values, test_errors, label='Test Error')
plt.gca().invert_xaxis()  # Reverse the x-axis to show decreasing values of k
plt.xlabel('K Values')
plt.ylabel('Error')
plt.title('KNN Train vs Test Error For Different k Values')
plt.legend()
plt.show()

In [ ]:
#most suitable k with minimum test error
k_suitable = k_values[np.argmin(test_errors)]
print('The most suitable k with the minimum test error is ', k_suitable)

# use the suitable k* as 4 to train the model
knn_ideal = KNeighborsClassifier(n_neighbors=k_suitable, metric='euclidean')
knn_ideal.fit(X_train, y_train)
suitable_prediction = knn_ideal.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
k_confusion_matrix = confusion_matrix(y_test, suitable_prediction)
print('Confusion Matrix\n', k_confusion_matrix)
tn = k_confusion_matrix[0, 0]
fp = k_confusion_matrix[0, 1]
fn = k_confusion_matrix[1, 0]
tp = k_confusion_matrix[1, 1]

true_pos_rate = tp / (tp + fn)
true_neg_rate = tn / (tn + fp)

precision = precision_score(y_test, suitable_prediction)
recall = recall_score(y_test, suitable_prediction)
f1 = f1_score(y_test, suitable_prediction)

print('True Positive Rate: ', true_pos_rate)
print('True Negative Rate: ', true_neg_rate)
print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)

## Part C iii.

In [ ]:
best_error_rates = [] # empty list initalized

# Values of N
N_values = range(10, 211, 20)  # Range of N values from 10 to 210 with a step of 20

for N in N_values:
    Class0_N = int(np.floor(N / 3))# subset of training set, 1st N/3 rows of Class 0
    Class1_N = N - Class0_N # 1st N - (N/3) rows of Class 1
    
    if Class0_N > len(train_class_0) or Class1_N > len(train_class_1):
        continue
        
    class0_subset = train_class_0.iloc[:Class0_N] # get the 1st N_class_0 rows from class0 df to form subset
    class1_subset = train_class_1.iloc[:Class1_N] # get the 1st N_class_1 rows from class1 df to form subset
    traindata_subset = pd.concat([class0_subset, class1_subset]) # combine class 0 and 1 subsets

    
    Xtrain_subset = traindata_subset.drop('Class_Label', axis=1) # split training data 
    Ytrain_subset = traindata_subset['Class_Label']

    kval_range = range(1, N, 5) # range of k values for each N going in increments of 5

    besttest_errorrate = float('inf') #set to infinity so any comparison with real error rates will result in smaller initial val 
    
    for k in kval_range:
        knn_new = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
        knn_new.fit(Xtrain_subset, Ytrain_subset)
        
        testset_prediction = knn_new.predict(X_test) # predict for test set
        
        errorrate = 1 - accuracy_score(y_test, testset_prediction) #error rate
    
        if errorrate < besttest_errorrate:
            besttest_errorrate = errorrate
    
    best_error_rates.append(besttest_errorrate) #update error rate

plt.figure(figsize=(8, 4)) # plot learning curve
plt.plot(N_values, best_error_rates, color='blue', label='Test Error')
plt.xlabel('N')
plt.ylabel('Best Test Error Rate for each N')
plt.title('Learning Curve')
plt.legend()
plt.show()

## Part D i. A)

In [ ]:
k_values_new = range(1, 197, 5) # k values {1,6,11,...,196}
results = [] 

manhattan_error = [] # initialize list for errors
for k in k_values_new:
    KNNMinkowski = KNeighborsClassifier(n_neighbors = k, metric = 'minkowski', p = 1) # knn manhattan classifier when p = 1
    KNNMinkowski.fit(X_train, y_train) #fit on training data
    predicted_vals = KNNMinkowski.predict(X_test) # use test data to find predicted values
    errorrate = 1 - accuracy_score(y_test, predicted_vals)
    manhattan_error.append(errorrate)

best_k_manhattan = k_values_new[np.argmin(manhattan_error)] # find the best k for manhattan metric by taking min of testing error
best_manhattan_error = min(manhattan_error) 
results.append(('Manhattan', best_k_manhattan, 1, best_manhattan_error))
errorresults_table = pd.DataFrame({      'Metric'       : 'Manhattan',
                                         'k'            : k_values_new, 
                                         'Test Error'   : manhattan_error})
print(errorresults_table)
print('The best k value for Manhattan distance when p = 1 is ', best_k_manhattan)

## Part D i. B)

In [ ]:
Kresults_log = []
log_p = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

log_error = []
for p in log_p:
    KNN_log = KNeighborsClassifier(n_neighbors = 6, metric = 'minkowski', p=p)
    KNN_log.fit(X_train, y_train)
    predicted_vals = KNN_log.predict(X_test)
    errorrate = 1 - accuracy_score(y_test, predicted_vals)
    log_error.append(errorrate)
    
best_k_manhattan = log_p[np.argmin(log_error)]
best_log_error = min(log_error)
Kresults_log.append(('Manhattan', best_k_manhattan, p, errorrate))

errorresults_table_log = pd.DataFrame({  'Metric'       : 'log10(p)',
                                         'log10(p)'     :  [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 
                                         'Test Error'   : log_error})
print(errorresults_table_log)
    
print('The best log10(p) is ', best_k_log)

## Part D i C) 

In [ ]:
k_values_new = range(1, 197, 5)
#initialize empty list for results and error rate
chebyshev_results = []
chebyshev_error = []

for k in k_values_new:
    KNN_Chebyshev = KNeighborsClassifier(n_neighbors=k, metric='chebyshev') #chebyshev metric classifier
    KNN_Chebyshev.fit(X_train, y_train) # fit on training data
    predicted_values = KNN_Chebyshev.predict(X_test) # predicted values based on test data
    errorrate = 1 - accuracy_score(y_test, predicted_values)
    chebyshev_error.append(errorrate)

best_k_chebyshev = k_values_new[np.argmin(chebyshev_error)] # find best k with min error from the range
best_chebyshev_error = min(chebyshev_error)
chebyshev_results.append(('Chebyshev', best_k_chebyshev, 'Infinity', best_chebyshev_error)) #add results to the list
errorresults_table_chebyshev = pd.DataFrame({  'Metric'       : 'Chebyshev',
                                         'k'            : k_values_new, 
                                         'Test Error'   : chebyshev_error})
print(errorresults_table_chebyshev)
print('The best k value for Chebyshev distance with p --> infinity is ', best_k_chebyshev)

## Part D ii.

In [ ]:
from numpy.linalg import inv

In [ ]:
mahalanobis_error = []

cov_matrix = np.cov(X_train, rowvar=False)
inverse_cov_matrix = inv(cov_matrix)

# Define the range for k
k_values_covariance = range(1, 197, 5)  # {1, 6, 11, ..., 196}

best_mahalanobis_error = float('inf')
best_k_mahalanobis = 0

for k in k_values_covariance: 
    KNN_Mahalanobis = KNeighborsClassifier(n_neighbors=k, metric='mahalanobis', metric_params={'VI': inverse_cov_matrix}) # Mahalanobis classifier with metric param = inverse covariance matrix
    KNN_Mahalanobis.fit(X_train, y_train) #fit on training data
    predicted_values = KNN_Mahalanobis.predict(X_test)
    errorrate = 1 - accuracy_score(y_test, predicted_values) #error rate
    mahalanobis_error.append(errorrate)
    
    if errorrate < best_mahalanobis_error: #if encountered error rate is less than the best error rate, then set mahalanobis error to error rate
        best_mahalanobis_error = errorrate
        best_k_mahalanobis = k
          
errorresults_table_mahalanobis = pd.DataFrame({  'Metric'       : 'Mahalanobis',
                                         'k'            : k_values_covariance, 
                                         'Test Error'   : mahalanobis_error})
print(errorresults_table_mahalanobis)
print('The best k value for Mahalanobis distance is ', best_k_mahalanobis)

## Part E

In [ ]:
k_values = range(1, 197, 5)

best_error_euclidean = float('inf') # set to infinity so 1st error rate will be less than initial value
best_k_euclidean = 0


for k in k_values:#find Euclidean distance
    KNN_Euclidean = KNeighborsClassifier(n_neighbors=k, metric='euclidean', weights='distance') # KNN Euclidean classifier with distance weight
    KNN_Euclidean.fit(X_train, y_train)
    predicted_vals = KNN_Euclidean.predict(X_test)
    errorrate = 1 - accuracy_score(y_test, predicted_vals)
    
    if errorrate < best_error_euclidean: # current error rate should be best one for euclidean metric
        best_error_euclidean = errorrate
        best_k_euclidean = k

print('Euclidean Distance\nBest K: ', best_k_euclidean, '\nBest Test Error:', best_error_euclidean)

In [ ]:
k_values = range(1, 197, 5)

best_error_manhattan = float('inf') # set to infinity so 1st error rate will be less than initial value
best_k_manhattan = 0

# Manhattan distance
for k in k_values:
    KNN_W_Manhattan = KNeighborsClassifier(n_neighbors=k, metric='minkowski', weights='distance', p = 1) # KNN Manhattan classifier with distance weight
    KNN_W_Manhattan.fit(X_train, y_train)
    predicted_vals = KNN_W_Manhattan.predict(X_test)
    errorrate = 1 - accuracy_score(y_test, predicted_vals)
    
    if errorrate < best_error_manhattan: # current error rate should be best 1 for manhattan metric
        best_error_manhattan = errorrate
        best_k_manhattan = k

print('Manhattan Distance\nBest K: ', best_k_manhattan, '\nBest Test Error:', best_error_manhattan)

In [ ]:
k_values = range(1, 197, 5)

best_error_chebyshev = float('inf') # set to infinity so 1st error rate will be less than initial value
best_k_chebyshev = 0

# chebyshev distance
for k in k_values:
    KNN_W_Chebyshev = KNeighborsClassifier(n_neighbors=k, metric='chebyshev', weights='distance', p = math.inf) # KNN Chebyshev classifier with distance weight
    KNN_W_Chebyshev.fit(X_train, y_train)
    predicted_vals = KNN_W_Chebyshev.predict(X_test)
    errorrate = 1 - accuracy_score(y_test, predicted_vals)
    
    if errorrate < best_error_chebyshev: # current error rate should be best 1 for manhattan metric
        best_error_chebyshev = errorrate
        best_k_chebyshev = k

print('Chebyshev Distance\nBest K: ', best_k_chebyshev, '\nBest Test Error:', best_error_chebyshev)

## Part F

In [ ]:
train_error_lowesteuclidean = min(train_errors)
train_error_lowestmanhattan = min(manhattan_error)
train_error_lowestchebyshev = min(chebyshev_error)
print(train_error_lowesteuclidean, train_error_lowestmanhattan, train_error_lowestchebyshev)

The lowest test error I achieved on this homework was 0.0 for Euclidean distance.

## References

https://seaborn.pydata.org/generated/seaborn.pairplot.html

https://seaborn.pydata.org/generated/seaborn.boxplot.html

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html

https://realpython.com/knn-python/

https://favtutor.com/blogs/infinity-python#:~:text=Python%20provides%20a%20predefined%20keyword,mathematical%20operations%20tends%20towards%20infinity.


https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

https://stackoverflow.com/questions/62847769/how-to-find-all-the-true-positives-negatives-and-false-positives-and-negatives-c

https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

https://stackoverflow.com/questions/48288047/scikit-learn-nearest-neighbor-search-with-weighted-distance-metric